# Data Cleaning with a Craigslist Automobile Dataset

In this notebook, we will use a number of fundamental data cleaning techniques in order to transform this raw Craigslist dataset to one that is ready to be processed and analyzed by machine learning algorithms. Let's see what we can apply to this dataset in order to extract some meaningful insights using data visualization.

The dataset I used can be found here: https://github.com/AMistry001/Data_Cleaning_With_Python-Craigslist_Cars/blob/main/vehicles.csv

First we import our basic dependencies; the Pandas and Numpy libraries will be sufficient for nearly all our data cleaning tasks, while Matplotlib and Seaborn will be used for some basic data visualization.

In [30]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [31]:
#Replace the file path with wherever you saved the file on your system
df = pd.read_csv('Desktop/vehicles.csv')

Let's take a look at the first 5 rows of our dataset.

In [32]:
df.head()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,drive,size,type,paint_color,image_url,description,county,state,lat,long
0,7184791621,https://duluth.craigslist.org/ctd/d/duluth-200...,duluth / superior,https://duluth.craigslist.org,6995,2000.0,gmc,new sierra 1500,excellent,8 cylinders,...,4wd,NaN,NaN,red,https://images.craigslist.org/00n0n_f06ykBMcdh...,2000 *** GMC New Sierra 1500 Ext Cab 157.5 WB...,NaN,mn,46.8433,-92.2550
1,7184773187,https://duluth.craigslist.org/cto/d/saginaw-20...,duluth / superior,https://duluth.craigslist.org,8750,2013.0,hyundai,sonata,excellent,4 cylinders,...,fwd,NaN,NaN,grey,https://images.craigslist.org/00d0d_kgZ6xoeRw2...,For Sale: 2013 Hyundai Sonata GLS - $8750. O...,NaN,mn,46.9074,-92.4638
2,7193375964,https://newhaven.craigslist.org/cto/d/stratfor...,new haven,https://newhaven.craigslist.org,10900,2013.0,toyota,prius,good,4 cylinders,...,fwd,NaN,NaN,blue,https://images.craigslist.org/00d0d_3sHGxPbY2O...,2013 Prius V Model Two. One owner—must sell my...,NaN,ct,41.1770,-73.1336
3,7195108810,https://albuquerque.craigslist.org/cto/d/albuq...,albuquerque,https://albuquerque.craigslist.org,12500,2003.0,mitsubishi,lancer,good,4 cylinders,...,4wd,mid-size,sedan,grey,https://images.craigslist.org/00m0m_4a8Pb6JbMG...,"2003 Mitsubishi Lancer Evolution, silver. Abo...",NaN,nm,35.1868,-106.6650
4,7184712241,https://duluth.craigslist.org/ctd/d/rush-city-...,duluth / superior,https://duluth.craigslist.org,16995,2007.0,gmc,sierra classic 2500hd,good,8 cylinders,...,4wd,full-size,truck,white,https://images.craigslist.org/01414_g093aPtSMW...,"**Bad Credit, No Credit... No Problem!**2007 G...",NaN,mn,45.6836,-92.9648


Let's also look at the size of the dataset, as well as the names of its comprising features.

In [33]:
print(df.shape)
df.columns

(423857, 25)


Index(['id', 'url', 'region', 'region_url', 'price', 'year', 'manufacturer',
       'model', 'condition', 'cylinders', 'fuel', 'odometer', 'title_status',
       'transmission', 'vin', 'drive', 'size', 'type', 'paint_color',
       'image_url', 'description', 'county', 'state', 'lat', 'long'],
      dtype='object')

We can immediately see that certain columns, such as "id", "url", "vin", "county", "description", "lat", "long", "model", "region_url", and "image_url", are irrelevant to our purposes. We can drop these using the ".drop()" function.

In [34]:
df.drop(columns=['id', 'url', 'image_url', 'region_url', 'vin', 'county', 'lat', 'long', 'description', 'region', 'model'], inplace=True)

In [35]:
df.head()

,price,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state
0,6995,2000.0,gmc,excellent,8 cylinders,gas,167783.0,clean,automatic,4wd,NaN,NaN,red,mn
1,8750,2013.0,hyundai,excellent,4 cylinders,gas,90821.0,clean,automatic,fwd,NaN,NaN,grey,mn
2,10900,2013.0,toyota,good,4 cylinders,hybrid,92800.0,clean,automatic,fwd,NaN,NaN,blue,ct
3,12500,2003.0,mitsubishi,good,4 cylinders,gas,NaN,clean,manual,4wd,mid-size,sedan,grey,nm
4,16995,2007.0,gmc,good,8 cylinders,diesel,254217.0,clean,automatic,4wd,full-size,truck,white,mn


That's looking much cleaner already. We can now turn our attention to the null values present in this dataset. As you may already be able to see, certain columns like "size", "type", and "odometer" contain "NaN" values. These must be dealt with, because null values are unable to be processed by machine learning algorithms.

Let's first obtain a count of how many null values are present per column.

In [36]:
df.isna().sum()

price                0
year             95114
manufacturer    110615
condition       247138
cylinders       226178
fuel             96643
odometer        153272
title_status     96098
transmission     95792
drive           192738
size            321230
type            182700
paint_color     201654
state                0
dtype: int64

There's a few ways we can deal with null values. We could replace them with another label, or with the column's mean, median, or mode, or we could drop them entirely.

Some of these columns, like "size" and "condition", simply contain too many null values. For the "size" column, for example, nearly 75% of the entries in the column are null values. In this case, we can go ahead and drop the columns.

In [37]:
df.drop(columns=['size','type', 'condition'], inplace=True)

In [38]:
df.isna().sum()

price                0
year             95114
manufacturer    110615
cylinders       226178
fuel             96643
odometer        153272
title_status     96098
transmission     95792
drive           192738
paint_color     201654
state                0
dtype: int64

For "paint_color", "title_status", "transmission", "fuel", and "drive", let's go ahead and replace the NaN values with the mode of its respective columns.

In [39]:
for column in ['paint_color', 'title_status', 'drive', 'transmission', 'fuel']:
    df[column].fillna(df[column].mode()[0], inplace=True)

In [40]:
df.isna().sum()

price                0
year             95114
manufacturer    110615
cylinders       226178
fuel                 0
odometer        153272
title_status         0
transmission         0
drive                0
paint_color          0
state                0
dtype: int64

For "odometer" and "cylinders", we can try replacing the NaN values with the mean. However, as we saw earlier, the values in our "cylinders" columns are strings, rather than numerical values. We can confirm this with the "dtypes" method.

In [41]:
df.dtypes

price             int64
year            float64
manufacturer     object
cylinders        object
fuel             object
odometer        float64
title_status     object
transmission     object
drive            object
paint_color      object
state            object
dtype: object

Indeed, our "cylinder" column is of type "object", which means we can't take the mean of it until we cast it to a numerical data type like int64 or float64. Let's take a look at our "cylinders" column again.

In [29]:
df.head()

,price,year,manufacturer,cylinders,fuel,odometer,title_status,transmission,drive,paint_color,state
0,6995,2000.0,gmc,8,gas,167783.000000,clean,automatic,4wd,red,mn
1,8750,2013.0,hyundai,4,gas,90821.000000,clean,automatic,fwd,grey,mn
2,10900,2013.0,toyota,4,hybrid,92800.000000,clean,automatic,fwd,blue,ct
3,12500,2003.0,mitsubishi,4,gas,99425.784611,clean,manual,4wd,grey,nm
4,16995,2007.0,gmc,8,diesel,254217.000000,clean,automatic,4wd,white,mn


Before we can cast our values to float, we need to remove the trailing "cylinders" tag. We can do this as follows:

In [45]:
df['cylinders'] = df['cylinders'].str.replace(' cylinders', '')

Let's check our "cylinders" column again.

In [46]:
df.head()

,price,year,manufacturer,cylinders,fuel,odometer,title_status,transmission,drive,paint_color,state
0,6995,2000.0,gmc,8,gas,167783.0,clean,automatic,4wd,red,mn
1,8750,2013.0,hyundai,4,gas,90821.0,clean,automatic,fwd,grey,mn
2,10900,2013.0,toyota,4,hybrid,92800.0,clean,automatic,fwd,blue,ct
3,12500,2003.0,mitsubishi,4,gas,NaN,clean,manual,4wd,grey,nm
4,16995,2007.0,gmc,8,diesel,254217.0,clean,automatic,4wd,white,mn


We can now cast these values to float64, as follows:

In [56]:
df['cylinders'] = pd.to_numeric(df['cylinders'],errors='coerce')
df.dtypes

price             int64
year            float64
manufacturer     object
cylinders       float64
fuel             object
odometer        float64
title_status     object
transmission     object
drive            object
paint_color      object
state            object
dtype: object

Now that our cylinder values are numeric, we can fill the NaN values in 'cylinders' and 'odometer' with the mean of the columns, rounding to the nearest whole number.

In [60]:
for column in ['cylinders', 'odometer']:
    df[column].fillna(df[column].mode().round()[0], inplace=True)

In [61]:
df.isna().sum()

price                0
year             95114
manufacturer    110615
cylinders            0
fuel                 0
odometer             0
title_status         0
transmission         0
drive                0
paint_color          0
state                0
dtype: int64

Lastly, for the 'year' and 'manufacturer' columns, we can elect to simply drop the rows containing NaN values.

In [64]:
df = df.dropna()

In [65]:
df.isna().sum()

price           0
year            0
manufacturer    0
cylinders       0
fuel            0
odometer        0
title_status    0
transmission    0
drive           0
paint_color     0
state           0
dtype: int64

We have now dealt with all irrelevant columns, and all null values. The data is now clean and is ready to be visualized or used by a machine-learning algorithm.